In [43]:
%load_ext autoreload
%autoreload 2

from webbrowser import get
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from models import *
import logging
from pretty_logger import get_logger
from pathlib import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
# define subset (ignore all other data)
ema = [f"Y{i}" for i in range(1, 8, 1)]
physical = [f"P{i}" for i in range(1, 5, 1)]
social = [f"S{i}" for i in range(1, 8, 1)]

In [45]:
df = pd.read_csv(datafile)
df["date"] = pd.to_datetime(df["day_survey"])
df["C"] = df["date"].apply(
    lambda date: (
        COVIDStatus.POST_COVID if date > date_covid else COVIDStatus.PRE_COVID
    )
)

df_head = df.head(5).copy()

df.rename(columns=reverse_ema_dictionary, inplace=True)
df.set_index(["uid", "date"], inplace=True)

subset = ema + physical + social + sleep + ["C"]
df = df[subset]
df.dropna(inplace=True)

sets_df = pd.read_parquet(sets_file, engine="pyarrow")

In [46]:
skip_emas = list(set(ema_dictionary.keys()).difference(set(["Y2", "Y3"])))
skip_treatments = list(
    set(subset).difference(
        set(
            [
                "P1",
                "P4",
                "S1",
                "S2",
                "S3",
                "S4",
                "S5",
                "S6",
                "Z1",
                "Z2",
                "Z3",
            ]
        )
    )
)

skip_treatments

['Y2', 'P2', 'P3', 'Y1', 'Y3', 'Y5', 'Y6', 'Y7', 'Y4', 'S7', 'C']

In [47]:
# index = 17
# model_row = sets_df.iloc[index]

already_fitted_sets = []
ccp_alpha = 0
n_estimatorss = [600]

for index, model_row in sets_df.iterrows():
    n_estimators = n_estimatorss[0]
    covariate_set = CovariateSet(
        row=model_row,
        data=df,
        outcomes_to_skip=skip_emas,
        treatments_to_skip=skip_treatments,
    )

    if not covariate_set.valid_set:
        logger.debug(f"Skipping {covariate_set} (no valid set)")
        continue

    if covariate_set.set_to_fit in already_fitted_sets:
        logger.debug(
            f"Skipping {covariate_set} (already did {covariate_set.set_to_fit})"
        )
        continue

    already_fitted_sets.append(covariate_set.set_to_fit)
    logger.info(f"Fitting\n{covariate_set!r}")
    logger.info(
        f"Median of {covariate_set.treatment} {df[covariate_set.treatment].median()}"
    )
    wbm = WBRandomForestModel(
        data=df,
        n_estimators=n_estimators,
        ccp_alpha=ccp_alpha,
        treatment=covariate_set.treatment,
        outcome=covariate_set.outcome,
        separating_set=covariate_set.restricted_adjustment_set,
        name=f"row:{index}",
    )
    logger.info(
        f"\nccp_alpha ={ccp_alpha}, n_estimators={n_estimators}, "
        f"pre_rsq train={wbm.pre_r_squared[0]}, pre_rsq test={wbm.pre_r_squared[1]}\n"
        f"pre mae test ={wbm.pre_r_squared[2]}\n"
        f"post_rsq train={wbm.pre_r_squared[0]}, post_rsq test={wbm.pre_r_squared[1]}\n"
    )
    logger.info("-----------------------------------------------------\n")

[2024-05-28 13:14:56.130] - modelslog- INFO N/A- 2000932153.py: 28 [<module>]: Fitting
treatment: P1:excercise (seconds), outcome: Y2:phq4_score, adjustment set={'S1', 'P2', 'S6', 'S3', 'S4', 'P4', 'S2'}
[2024-05-28 13:14:56.132] - modelslog- INFO N/A- 2000932153.py: 29 [<module>]: Median of P1 10963.4


[2024-05-28 13:15:15.704] - modelslog- INFO N/A- 2000932153.py: 41 [<module>]: 
ccp_alpha =0, n_estimators=600, pre_rsq train=0.8814502361879364, pre_rsq test=0.17779542262851178
pre mae test =1.7744972025058474
post_rsq train=0.8814502361879364, post_rsq test=0.17779542262851178

[2024-05-28 13:15:15.704] - modelslog- INFO N/A- 2000932153.py: 47 [<module>]: -----------------------------------------------------

[2024-05-28 13:15:15.705] - modelslog- INFO N/A- 2000932153.py: 28 [<module>]: Fitting
treatment: P4:sports (hours), outcome: Y2:phq4_score, adjustment set={'S1', 'P2', 'S5', 'S3', 'P3', 'S4'}
[2024-05-28 13:15:15.706] - modelslog- INFO N/A- 2000932153.py: 29 [<module>]: Median of P4 0.08325
[2024-05-28 13:15:37.896] - modelslog- INFO N/A- 2000932153.py: 41 [<module>]: 
ccp_alpha =0, n_estimators=600, pre_rsq train=0.8709023067330046, pre_rsq test=0.10318332236451166
pre mae test =1.7672244665859387
post_rsq train=0.8709023067330046, post_rsq test=0.10318332236451166

[2024-05-

KeyboardInterrupt: 